In [ ]:
import csv

# Global Customer list
customers = []

In [ ]:
#Account class
class Account:
    """
    Base account class used by Checking, Savings, and Credit.

    Attributes:
    account_id: four-digit account ID as string
    balance: current balance of the account (for credit accounts shows amount owed)
    interest: rate applies to account

    """
    def __init__(self,account_id = "0000",balance = 0.0,interest = 0):
        """
        Initialize a general account

        """
        self.account_id = account_id
        self.balance = balance
        self.interest = interest

    def __str__(self):
        """
        Print account data neatly
        """
        return f"ID: {self.account_id}, Balance: ${self.balance}, Interest: {self.interest*100}%"
    
    # Get and Set methods for account ID

    def get_account_id(self):
        """
        Return the account ID

        """
        print("getting account ID")
        return self.account_id
    
    def set_account_id(self,value):
        """
        Set the account ID
        """
        print("setting account id to", value)

        if not isinstance(value,str) and len(value) != 4:
            raise ValueError("Account ID must be a 4-digit string")
        
        self.account_id = value
    
    account_id = property(get_account_id, set_account_id)

    # Get and Set methods for balance
    
    def get_balance(self):
        """
        Return the current balance

        """
        print("getting balance")
        return self.balance
    
    def set_balance(self,value):
        """
        Set the account balance

        """
        print("setting balance to", value)

        if not isinstance(value,(int,float)):
            raise TypeError("Balance mist be a number")
        
        self.balance = float(value)

    balance = property(get_balance, set_balance)

    # Core methods

    def deposit(self,amount):
        """
        Add money to the account

        """
        self.balance += amount

    def withdraw(self,amount):
        """
        Remove money from the account if there is enough funds

        """
        if amount > self.balance:
            print("Withdrawal denied: insufficient funds")
            return False
        self.balance -= amount
        return True
    

In [ ]:
class Checking(Account):
    """
    Checking account class
    Interest rate is always 0%

    """

    def __init__(self,account_id, balance):
        super().__init__(account_id, balance, interest= 0)

In [ ]:
class Savings(Account):
    """
    Savings account class
    Interest rate is always 1%

    """

    def _init__(self,account_id, balance):
        super().__init__(account_id, balance, interest = 0.01)

In [ ]:
class Credit(Account):
    """
    Credit account class. 

    Attributes:
    credit_limit: Maximum amount allowed to be charged

    """

    def __init__(self, account_id, balance, credit_limit):
        super().__init__(account_id, balance, interest= 0.30)
        self.credit_limit = credit_limit

# Get and Set methods for credit limit

def get_credit_limit(self):
    """
    Return credit limit

    """

    print

In [ ]:
class Customer:
    ...